In [2]:
from scipy.io import savemat
from tempfile import TemporaryFile
import numpy as np
import itertools as it
from scipy.integrate import quad
import math as mt
import scipy.special as psi
import matplotlib.pyplot as plt
plt.style.use('classic')
import seaborn as sns
import pandas as pd
from scipy.io import loadmat
from scipy import stats
from numpy.random import seed
from numpy.random import rand
import math as mth
import matplotlib.gridspec as gridspec
blue=(0.00784313725490196, 0.24313725490196078, 1.0)
red=(0.9098039215686274, 0.0, 0.043137254901960784)

In [3]:
# Integral function for the compuation of entropy to reduce bias
def integrand(t, ni):
     return (t**(ni-1))/(1+t)

In [4]:
# Computes all the permutations of a range 0-k
def permutation(k):
  f_k=mt.factorial(k)
  A=np.empty((f_k,k))
  for i, perm in enumerate(it.permutations(range(k))):
      A[i,:] = perm
  return A

In [5]:
# Converts an array into a list
def array_list(array_num): 
    num_list = array_num.tolist()
    return num_list

In [6]:
# Creates an array that displays which permutation represent the order
# of the input array
  
def ubble(v):
    n=len(v)
    a=range(0,n)
    B=np.array([v,a])
    t=np.array([0., 0.])
    for i in range(0,n-1):
        for j in range(1,n):
            if B[0,j-1]>B[0,j]:
                
                t[:]=B[:,j]
                B[:,j]=B[:,j-1]
                B[:,j-1]=t[:]    
    ord=B[1][:]
    return ord

In [7]:
#Computes the mPE
    
# [H,h]=mPE(data,n)
# returns the permutation entropy (of order n) and normalised 
# entropy vector of a time series "data"
    
# [H,h]=mPE(data,n,eps)
# returns the permutation entropy (of order n) and normalised 
# entropy vector of a time series "data". The vector eps contains
# as many elements as the number of scales considered by the algorithm.
# Each element of the vector eps represents how many elements are 
# considered in creating a coarse-grained time series. Default is eps=[1]
    
# [H,h]=mPE(data,n,eps,L)
# returns the permutation entropy (of order n) and normalised 
# entropy vector of a time series "data". The vector eps contains
# as many elements as the number of scales considered by the algorithm.
# Each element of the vector eps represents how many elements are 
# considered in creating a coarse-grained time series. Default is eps=[1]
# L is a lag parameter, default is 1.
# 
# n(order of the entropy)
# L(distance between two adjacent elements in the computation)(generally=1)
def mPE(*argv):
    
    g=len(argv)
    
    if g==2:
        n=argv[1]
        datain=argv[0]
        L=1
        epsin=np.array([1])
    if g==3:
        n=argv[1]
        datain=argv[0]
        L=1
        epsin=argv[2]
    if g==4:
        n=argv[1]
        datain=argv[0]
        L=argv[3]
        epsin=argv[2]
    
    
    
#    Check for the data to be in the right dimension
    if len(datain[:,1])>len(datain[1,:]):
        datain=datain.transpose()
        
    
    scalesno=len(epsin)
    HH=np.zeros((1,scalesno));
    norm_HH=np.zeros((1,scalesno));
#    Definition of parameters: fac is the number of possible permutations
#    Elem is the No of data points
#    Dim is the dimensionality of the samples
    
    
    for eps in range(0,scalesno):
        scale=epsin[eps]
        xlen=len(datain[1,:])
        ylen=np.round((xlen/scale)-0.5)
        ylen=ylen.astype(int)
        D=np.zeros((len(datain[:,1]),ylen))
        for ylenc in range(0,ylen):
            dfg=datain[:,((ylenc)*scale):((ylenc+1)*scale)]
            fhk=np.sum(dfg,axis=1)
            r=(1/scale)*fhk
            D[:,ylenc]=r
            
        data=D
        fac=mt.factorial(n)
        elem=len(data[1,:])
        dim=len(data[:,1])
        
    #    A is a n-by-factorial(n) matrix that shows all the 
    #    possible permutations of n elements
        A=permutation(n)
        
    #    counter is a factorial(n) square matrix that counts the recurrence of
    #    a dim-dimensional permutation
        lpi=fac*np.ones((dim))
        lpi=lpi.astype(int)
        nj = lpi.tolist()
        counter=np.zeros((nj))
        
    #    For each iteration i, a series of n points is sampled from the 
    #    data set and the corresponding permutation is identified. 
    #    The counter matrix keeps track of the numiber of times a certain
    #    permutation, or combination of permutations is observed. 
        for i in range(0, elem-n*L+L-1):
            coord=np.zeros((dim))
            for num in range(0,dim):
                sample=data[num,range(i,i+L*(n),L)]
                ord=ubble(sample)
                perm_num=0
                check_1=1
                check_2=1
                
                while (perm_num<=fac) and (check_2):
                   
                    check_1=1
                    for j in range(0,n-1):
                        if ord[j]!=A[perm_num,j]:
                            check_1=0
                    if check_1:
                        coord[num]=perm_num
                        check_2=0
                        
                    perm_num=perm_num+1
            
                
            coord=coord.astype(int)
            coord1=tuple(coord)
            counter[coord1]=counter[coord1]+1
            
    #    Once the counter matrix is complete, each element is divided by the
    #    total number of samples to get a empirical probability, and the PE is
    #    computed according to the method described in Schurmann
        
        H=0
        counter1=counter.flatten()
        for iter in range(0,(fac**dim)):
           
            rec=counter1[iter]
            tot=elem-n+1
            
            if rec==0:
                H=H
                
            else:
                I = quad(integrand, 0, 1, args=(rec))
                I=I[0]
                coeff=((-1)**rec)*I
                prob=(rec/tot)*(psi.digamma(tot)-psi.digamma(rec)-coeff)
                H=H+mt.log2(mt.exp(prob))
             
    #     The normalised entropy norm_H is computed as well
        norm_H=H/(mt.log2(fac**dim))
        
        HH[0,eps]=H
        norm_HH[0,eps]=norm_H
    return [HH,norm_HH]

In [8]:
#Moving average function
def movav(data,n):
    n=n+1
    vo=np.zeros([len(data[:,0])-n+1,len(data[0])]);

    for i in range(0,len(data[:,0])-n+1):
        vec=data[i:i+n-1];
        vo[i]=np.mean(vec, axis=0);
    return(vo)

In [ ]:
#Load data, assemble in one array and sav them

res2eeg = 1250/20000
spks2tracking = 1/32
names=['mdm81-3011-0126','mdm90-1901-0127','mdm96-1606-0120','mdm96-1706-0120','mdm183-170218','mme08-161115', 'mme09-161205','mme37-200321','mme37-200322','mme37-200323','mme37-200324'] 

which=[[2, 4, 5, 7, 8, 10], [2, 4, 5, 7, 8, 10], [2, 8, 6, 2, 8, 6], [2, 8, 4,  2, 8, 6], [1, 6, 3, 4, 2, 5], [1, 6, 3, 4, 2, 5], [1, 6, 3, 4, 2, 5], [2, 9, 5, 6, 4, 8], [2, 9, 5, 6, 4, 8], [2, 9, 5, 6, 4, 8], [2, 9, 5, 6, 4, 8]];
data_not_nan=[None]*11

for index in range(0,11):
    name=names[index]
    where=which[index]
    l = [None] * 6
    for f in range (0,len(where)):
        
        hh=where[f];
        speed = loadmat(name+'_'+str(hh)+'.mat')
        track=speed['track']
        track_not_nan = track[~np.isnan(track[:,0]) & ~np.isnan(track[:,1]),:]
        
        if index>1 and index<4 and f>2:
            track_not_nan = np.random.rand(1000,2)
        
        l[f]=track_not_nan
        
        
        
    data_not_nan[index]=l

savemat('data_all_nonan.mat', mdict={'list':data_not_nan})

In [ ]:
#Define number of bins to divide each session in, order of movav filter
bins=40
filter_order=35

In [ ]:
#Divide in bins and movav filter, rescales sessions

data_all_nonan = loadmat('data_all_nonan.mat')
data_all_nonan= data_all_nonan['list']

M=[None]*11

for i in range (0,11):
    M1=[None]*6
    for j in range (0,6):
        S=data_all_nonan[i][j]
        S=movav(S,filter_order)
        S=S-np.amin(S,axis=0)
        S=S/np.amax(S,axis=0)
        
        if i==6 and j==3:
            S=S[S[:,0]>0.38]
            S=S-np.amin(S,axis=0)
            S=S/np.amax(S,axis=0)
            
        if i>3 and j>3:
            S[:,0]=S[:,0]*2
            
        leng=len(S[:,0])
        lin=np.linspace(0,leng-1,num=bins+1);
        lin=np.round(lin)
        
        F=[None]*(bins)
        for k in range(0,bins):
            d=S[int(lin[k]):int(lin[k+1]-1)]
            F[k]=d
        M1[j]=F
    M[i]=M1

savemat('data_fil35_bins40.mat', mdict={'list':M})

In [ ]:
#Choose orders of entropy to compute
#From:
from_entr=3
#To
to_entr=4

In [ ]:
#Compute Entropy and average speed for each bins considered
data = loadmat('data_fil35_bins40.mat')
data= data['list']

E=[None]*11
S=[None]*11
for aa in range(0,11):
    E1=[None]*6
    S1=[None]*6
    for bb in range(0,6):
        A=data[aa][bb]
        v_vect=np.zeros([1,len(A)])
        en_vect=np.zeros([3,len(A)])
        for k in range(0,len(A)):
            B=A[k]
            
            
            x=B[:,0]  
            y=B[:,1]
            speed=np.zeros([1,len(x)-1])
            for nn in range(0,len(x)-1):
                vx=(x[nn+1]-x[nn])*0.4/0.0256
                vy=(y[nn+1]-y[nn])*0.4/0.0256
                v=(vx**2+vy**2)**0.5
                speed[0,nn]=v
            v_vect[0,k]=np.mean(speed,axis=1)
            
            
            
            for ordine in range(from_entr,to_entr+1):
                [HH,hh]=mPE(B,ordine)
                en_vect[ordine-2,k]=HH
            E1[bb]=en_vect
            S1[bb]=v_vect
        E[aa]=E1
        S[aa]=S1
        
savemat('Entr_Speed_35_40.mat', mdict={'entr':E,'speed':S})

In [ ]:
#Choose order of entropy to analyse
order=4;
order=order-2;

In [ ]:
#Assemble data in a pandas dataset to be used for graphical visualisation
a=loadmat('Entr_Speed_35_40.mat')
stre=a['speed']
etre=a['entr']


etre2=np.zeros([2640])
stre2=np.zeros([2640])
for i in range(0,11):
    for ii in range(0,6):
        for iii in range(0,40):
            etre2[240*i+40*ii+iii]=etre[i][ii][order][iii]
            stre2[240*i+40*ii+iii]=stre[i][ii][0][iii]*100
            
g=np.ones([2640])
g[480:960]=g[480:960]*2;
g[960:1680]=g[960:1680]*3
g[1680:2640]=g[1680:2640]*4
g[600:720]=g[600:720]*2.5;
g[840:960]=g[840:960]*2.5;



h=np.ones([2640])*2
h[np.r_[0:80,0+240:80+240,0+480:80+480,0+720:80+720,0+960:80+960,0+1200:80+1200,0+1440:80+1440,0+1680:80+1680,0+1920:80+1920,0+2160:80+2160,0+2400:80+2400]]=h[np.r_[0:80,0+240:80+240,0+480:80+480,0+720:80+720,0+960:80+960,0+1200:80+1200,0+1440:80+1440,0+1680:80+1680,0+1920:80+1920,0+2160:80+2160,0+2400:80+2400]]/2
k=rand(2640)

g1=[None]*2640
h1=[None]*2640

c=["One", "Two", "Three", "Four", "No"]
d=["Fam", "Nov"]

for i in range(0,2640):
    for j in range(0,5):
        if h[i]==j+1:
            h1[i]=d[j]
        if g[i]==j+1:
            g1[i]=c[j]


data={'Cond':h1,'Exp':g1,'Entr':etre2,'Speed':stre2, 'Casual':k}
df=pd.DataFrame(data)

   
df = df[df.Exp != 'No']
df = df[df.Entr != 0]

In [ ]:
##EXAMPLE OF DATA VISUALISATION
lblsz=28
tkssz=24
ttlsz=30
lgnsz=28

##Regression 2
fig, axes = plt.subplots(2,4, sharex=False,sharey=False)
fig.set_size_inches(20,10)

sns.regplot(x="Entr", y="Speed", data=df[(df.Exp == 'One')], order=2, ax=axes[0,0], marker='.', scatter_kws={'color': red},line_kws={'color': 'black'})
